In [1]:
from pathlib import Path

import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
moves_str = Path("../moves.txt").read_text()

In [90]:
next(filter(lambda s: "@" in s,moves_str.split("\n")))

'1 e4 d5 2 exd5 Qxd5 3 Nc3 Qd8 4 d4 Nf6 5 Be3 e6 6 Nf3 Nc6 7 P@e5 Bb4 8 exf6 Qxf6 9 N@e4 Qd8 10 Bd2 O-O 11 Nb5 Bxd2+ 12 Qxd2 h6 13 Be2 a6 14 O-O-O axb5 15 B@c5 Rxa2 16 Bxf8 Qxf8 17 b3 B@a3+ 18 Kb1 Rb2+ 19 Ka1 Nb4 20 Rb1 Nxc2+ 21 Qxc2 Rxc2 0-1'

In [3]:
print(moves_str[:500])

1 g3 d5 2 Bg2 e6 3 Nc3 f5 4 d4 Bd6 5 e4 Nf6 6 exd5 Nc6 7 dxc6 Bd7 8 Qe2 Bxc6 9 d5 Qd7 10 dxc6 Qxc6 11 Bxc6+ bxc6 12 Qxe6+ Kf8 13 Bg5 Be7 14 Nf3 h6 15 Ne5 Nd5 16 Qf7# 1-0
1 Nf3 Nf6 2 e4 Nxe4 3 c3 Nf6 4 Bc4 d6 5 Bxf7+ Kxf7 6 Qb3+ Ke8 7 Ng5 e6 8 Nxe6 Kf7 9 Ng5+ Ke8 10 Qf7# 1-0
1 e4 Nf6 2 Nf3 e6 3 e5 Be7 4 exf6 O-O 5 Be2 Qe8 6 O-O Bxf6 7 d4 Be5 8 c3 Bxh2+ 9 Kxh2 d6 10 Kg1 f6 11 d5 Qh5 12 dxe6 Qh1+ 13 Kxh1 Bxe6 14 Nd4 Bf7 15 Nf5 h6 16 Ne7+ Kh8 17 Bd3 Bg6 18 Bc2 Bh7 19 Bxh7 Kxh7 0-1
1 d4 Nf6 2 e3 Ne4 


In [4]:
block_size = 350

In [5]:
padded_moves = ""
for moves in moves_str.split("\n"):
    padded_moves += moves + (block_size - len(moves)) * " "

In [7]:
len(padded_moves) // block_size

1000274

In [51]:
batch_size = 64  # how many independent sequences will we process in parallel?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_head = 6
n_embd = n_head * 32  # 64
n_layer = 3
dropout = 0.2

In [10]:
torch.manual_seed(742)

# here are all the unique characters that occur in this text
chars = sorted(list(set(padded_moves)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

In [89]:
stoi

{' ': 0,
 '#': 1,
 '+': 2,
 '-': 3,
 '/': 4,
 '0': 5,
 '1': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 '=': 15,
 '@': 16,
 'B': 17,
 'K': 18,
 'N': 19,
 'O': 20,
 'P': 21,
 'Q': 22,
 'R': 23,
 'a': 24,
 'b': 25,
 'c': 26,
 'd': 27,
 'e': 28,
 'f': 29,
 'g': 30,
 'h': 31,
 'x': 32}

In [13]:
# Train and test splits
data = torch.tensor(encode(padded_moves), dtype=torch.long)
n = int(0.9 * (len(data) //block_size)) * block_size  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) // block_size - 1, (batch_size,)) * block_size
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [19]:
class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)  # (B,T,hs)
        q = self.query(x)  # (B,T,hs)
        v = self.value(x)  # (B,T,hs)
        # compute attention scores ("affinities")
        wei = (
            q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        )  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)        
        return out

In [20]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [21]:
class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [22]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [23]:
class ChessGPT(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [30]:
model = ChessGPT()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

1.413153 M parameters


In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [52]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.6973, val loss 0.6989
step 500: train loss 0.6300, val loss 0.6339
step 1000: train loss 0.5942, val loss 0.5969
step 1500: train loss 0.5553, val loss 0.5543
step 2000: train loss 0.5300, val loss 0.5278
step 2500: train loss 0.5105, val loss 0.5097
step 3000: train loss 0.4963, val loss 0.4957
step 3500: train loss 0.4832, val loss 0.4832
step 4000: train loss 0.4718, val loss 0.4767
step 4500: train loss 0.4673, val loss 0.4695
step 4999: train loss 0.4584, val loss 0.4597


In [75]:
prompt_moves = "1 d4 "

In [76]:
context = torch.tensor(encode(prompt_moves), dtype=torch.long, device=device).reshape(1,-1)
res = decode(m.generate(context, max_new_tokens=block_size - len(prompt_moves))[0].tolist())
print(res)

1 d4 d5 2 c4 Nc6 3 Nc3 a6 4 Nf3 f6 5 e4 e6 6 e5 Be7 7 Bd3 h5 8 Be2 h4 9 Nxh4 g5 10 h3 Bf8 11 Ng3 c6 12 Bc2 Qd7 13 Qd2 Bd7 14 O-O Ke7 15 f3 e5 16 f4 Nd4 17 Qe2 exd4 18 Bf3 B@b8 19 Nhg2 hxg3 20 Qxh8=Q Bxc4 21 Qxh8 exf4 22 Bf3 Qd2 23 Rxh4 Rh8 24 Nd2 Nxe5 25 dxe5 exf3 26 Nd3 Qe6 0-1                                                                       


In [77]:
prompt_moves in moves_str

True

In [78]:
for i in range(len(res)):
    if res[:i] not in moves_str:
        break

In [79]:
i

29

In [88]:
res

'1 d4 d5 2 c4 Nc6 3 Nc3 a6 4 Nf3 f6 5 e4 e6 6 e5 Be7 7 Bd3 h5 8 Be2 h4 9 Nxh4 g5 10 h3 Bf8 11 Ng3 c6 12 Bc2 Qd7 13 Qd2 Bd7 14 O-O Ke7 15 f3 e5 16 f4 Nd4 17 Qe2 exd4 18 Bf3 B@b8 19 Nhg2 hxg3 20 Qxh8=Q Bxc4 21 Qxh8 exf4 22 Bf3 Qd2 23 Rxh4 Rh8 24 Nd2 Nxe5 25 dxe5 exf3 26 Nd3 Qe6 0-1                                                                       '